In [1]:
# import required dependencies
import json
import numpy as np
import faiss
from openai import OpenAI
from sentence_transformers import SentenceTransformer
import time

c:\Users\lwinh\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
recipe_path = "files/outputs/recipes.jsonl"
index_path = "files/outputs/recipes-index-all-MiniLM-L6-v2.bin"
embedding_model = "all-MiniLM-L6-v2"
openai_api_key = "add-your-open-AI-key"

llm_model = "gpt-4.1-mini"      # good quality/cost;
top_k = 8                       # retrieve candidates
final_k = 3                     # keep best few for the LLM

In [3]:
# create open AI cilent
client = OpenAI(api_key=openai_api_key)

In [4]:
# load index
index = faiss.read_index(index_path)

In [5]:
# load recipes
recipes = {}
metadatas = []

with open(recipe_path, "r", encoding="utf-8") as f:
    for line in f:
        recipe = json.loads(line)
        recipes[recipe["id"]] = recipe["summary"]
        metadatas.append({ "id": recipe["id"], "title": recipe["title"]} )

In [6]:
model = SentenceTransformer(embedding_model)

In [7]:
# retrieve results
def retrieve(query, k = top_k):
    qv = model.encode([query], convert_to_numpy=True, normalize_embeddings=True).astype(np.float32)
    D, I = index.search(qv, k)

    hits = []
    for score, idx in zip(D[0], I[0]):
        metadata = metadatas[idx]
        hits.append({"score": float(score), "id": metadata["id"], "title": metadata["title"]})

        if len(hits) >= k:
            break

    return hits

# build context
def build_context_blocks(hits):
    blocks = []
    for h in hits:
        label = f"[{h["title"]} {h["id"]}]"
        blocks.append(f"{label}\n{recipes.get(h["id"])}")
    return "\n\n".join(blocks)

In [8]:
SYSTEM_PROMPT = """You are a culinary assistant. Answer ONLY using the provided recipe context.
If the answer isn't contained in the context, say you don't have that information.
When the user asks for a recipe, return:
- A short title
- Ingredients (bullets)
- Numbered steps
Always include citations as [Title • RecipeID] for any facts you use."""

def answer_with_rag(user_query, llm_model, k = top_k):
    # 1) Retrieve top candidates
    hits = retrieve(user_query, k)
    hits = hits[:final_k]
    # 2) Build context block
    context = build_context_blocks(hits)
    # 3) Call OpenAI Responses API with the context
    prompt = f"""User question: {user_query}

Context:
{context}
"""

    resp = client.responses.create(
        model = llm_model,
        input = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,  # keep it grounded
    )

    # 4) Extract text
    text = resp.output_text

    # Optional: return the sources we used for UI citations
    sources = [{"title": h["title"],
               "recipe_id": h["id"],
               "score": h["score"]} for h in hits]

    return text, sources

In [ ]:
# this is to test the query with LLM
if __name__ == "__main__":
    q = input("Please enter your query.\n")
    answer, sources = answer_with_rag(q, llm_model, 3)
    print("=== ANSWER ===")
    print(answer)
    print("\n=== SOURCES ===")
    for s in sources:
        print(s)

### Below codes are to evaluate process time for two LLM models

In [ ]:
queries = [
    "How to make Pumpkin Bars with Microwave?",
    "Show me how to cook Cold Oven Pound Cake with Chocolate.",
    "How to make banana oatmeal cookies? I have allergy.",
    "How to make banana oatmeal cookies for people with diabetic",
    "How to cook indian beef curry?",
    "How to cook fish chowder with oven?"
]

start_time = time.time()
for q in queries:
    answer, sources = answer_with_rag(q, llm_model, 3)

    print(answer)
end_time = time.time()

time_taken = end_time - start_time
print(time_taken)

Pumpkin Bars (Microwave)

Ingredients:
- Butter
- Brown sugar
- Eggs
- Vanilla
- Pumpkin
- Flour
- Baking powder
- Salt
- Baking soda
- Cinnamon
- Nutmeg
- Ginger

Steps:
1. Microwave butter for 30 seconds to melt.
2. Blend in brown sugar.
3. Add eggs, one at a time, beating after each addition.
4. Blend in remaining ingredients; mix smooth.
5. Lightly grease bottom only of a 12 x 8 x 2-inch pan.
6. Spread mixture evenly.
7. Microwave for 8 to 9 1/2 minutes at High, shielding bars after 4 minutes and turning dish every 2 minutes.
8. Cool.
9. Spread icing on bars and sprinkle on nuts.
10. Makes 24 bars. 

[Pumpkin Bars(Microwave) • 54c783a9-3e89-4d2d-8460-91bb02889bbb]
Cold Oven Chocolate Pound Cake

Ingredients:
- Sugar
- Eggs
- Flour
- Cocoa
- Butter (or oleo/shortening as variations)
- Baking powder
- Milk
- Vanilla

Directions:
1. Cream butter (or oleo/shortening) and sugar together.
2. Add eggs one at a time, beating well.
3. Sift and mix flour, baking powder, and cocoa.
4. Add van

In [ ]:
llm_model = "gpt-4o-mini"

queries = [
    "How to make Pumpkin Bars with Microwave?",
    "Show me how to cook Cold Oven Pound Cake with Chocolate.",
    "How to make banana oatmeal cookies? I have allergy.",
    "How to make banana oatmeal cookies for people with diabetic",
    "How to cook indian beef curry?",
    "How to cook fish chowder with oven?"
]

start_time = time.time()
for q in queries:
    answer, sources = answer_with_rag(q, llm_model, 3)

    print(answer)
end_time = time.time()

time_taken = end_time - start_time
print(time_taken)

### Pumpkin Bars (Microwave)

**Ingredients:**
- butter
- brown sugar
- eggs
- vanilla
- pumpkin
- flour
- baking powder
- salt
- baking soda
- cinnamon
- nutmeg
- ginger

**Steps:**
1. Microwave butter for 30 seconds to melt.
2. Blend in brown sugar.
3. Add eggs, one at a time, beating after each addition. Blend in remaining ingredients; mix smooth.
4. Lightly grease the bottom only of a 12 x 8 x 2-inch pan.
5. Spread the mixture evenly in the pan.
6. Microwave for 8 to 9 1/2 minutes at High, shielding bars after 4 minutes and turning the dish every 2 minutes.
7. Cool. Spread icing on bars and sprinkle on nuts.

Makes 24 bars. [Pumpkin Bars(Microwave) • 54c783a9-3e89-4d2d-8460-91bb02889bbb]
### Cold Oven Chocolate Pound Cake

**Ingredients:**
- sugar
- eggs
- flour
- cocoa
- butter
- baking powder
- milk
- vanilla

**Steps:**
1. Cream butter and sugar together.
2. Add eggs, flour, and baking powder.
3. Add cocoa, milk, and vanilla; mix well.
4. Pour the batter into a greased and well-